A notebook for practicing MAFFT (Multiple Alignment using Fast Fourier Transform).  The manual page is here: https://mafft.cbrc.jp/alignment/software/manual/manual.html

In [105]:
! mafft --version
! lsb_release -d 

v7.490 (2021/Oct/30)
Description:	Ubuntu 20.04.2 LTS


# Looking at the files

The fasta files I am practicing with are the 27 that passed hi_qc, from `result.illumina.20210513` (see the second commit of filter_fa_qc.sh for details on their extraction).  The files are:

In [106]:
! ls practice_fastas/*.consensus.fa

practice_fastas/COG111-E-NC_S83.primertrimmed.consensus.fa
practice_fastas/COG111-NC_S89.primertrimmed.consensus.fa
practice_fastas/COG111-PC_S86.primertrimmed.consensus.fa
practice_fastas/COG112-PC_S114.primertrimmed.consensus.fa
practice_fastas/NORW-220299_S80.primertrimmed.consensus.fa
practice_fastas/NORW-22A50F_S96.primertrimmed.consensus.fa
practice_fastas/NORW-22AE40_S97.primertrimmed.consensus.fa
practice_fastas/NORW-22D87C_S81.primertrimmed.consensus.fa
practice_fastas/NORW-22DDC8_S95.primertrimmed.consensus.fa
practice_fastas/NORW-22DFFF_S94.primertrimmed.consensus.fa
practice_fastas/NORW-230B31_S87.primertrimmed.consensus.fa
practice_fastas/NORW-23264D_S112.primertrimmed.consensus.fa
practice_fastas/NORW-23267A_S106.primertrimmed.consensus.fa
practice_fastas/NORW-232689_S137.primertrimmed.consensus.fa
practice_fastas/NORW-2326A7_S131.primertrimmed.consensus.fa
practice_fastas/NORW-2326C5_S143.primertrimmed.consensus.fa
practice_fastas/NORW-2326F2_S128.primertrimmed.consensus

Let's take a look at a random file:

In [107]:
! head practice_fastas/NORW-220299_S80.primertrimmed.consensus.fa

>Consensus_NORW-220299_S80.primertrimmed.consensus_threshold_0.75_quality_20
...


Check how many As, Cs, Ts, Gs and Ns there are, and how long the sequence is:

In [108]:
! seqkit stats practice_fastas/*.fa

file                                                         format  type     num_seqs  sum_len  min_len  avg_len  max_len
practice_fastas/COG111-E-NC_S83.primertrimmed.consensus.fa   FASTA   Unlimit         1        0        0        0        0
practice_fastas/COG111-NC_S89.primertrimmed.consensus.fa     FASTA   Unlimit         1        0        0        0        0
practice_fastas/COG111-PC_S86.primertrimmed.consensus.fa     FASTA   DNA             1   29,903   29,903   29,903   29,903
practice_fastas/COG112-PC_S114.primertrimmed.consensus.fa    FASTA   DNA             1   29,903   29,903   29,903   29,903
practice_fastas/NORW-220299_S80.primertrimmed.consensus.fa   FASTA   DNA             1   29,874   29,874   29,874   29,874
practice_fastas/NORW-22A50F_S96.primertrimmed.consensus.fa   FASTA   DNA             1   29,886   29,886   29,886   29,886
practice_fastas/NORW-22AE40_S97.primertrimmed.consensus.fa   FASTA   DNA             1   29,874   29,874   29,874   29,874
practice_fastas/

So, these are all about 29,900 bp long (right for SARS-CoV-2!), except that five files are empty except for the header, like the first one:

In [109]:
! head -10 practice_fastas/COG111-E-NC_S83.primertrimmed.consensus.fa

>Consensus_COG111-E-NC_S83.primertrimmed.consensus_threshold_0.75_quality_20



The empty files should not be included in the analysis; I will update `filter_fa_qc.sh` to make sure the files each contain at least one sequence.  Alternatively: just do the concatenation/filtering step as part of the bash script.  (Update: this has been done)

In [110]:
#testing out ways of how to do this

#count lines of text in fasta with one sequence
! cat practice_fastas/NORW-232BD5_S138.primertrimmed.consensus.fa | sed '/^\s*$/d' | wc -l
#count lines of text in empty fasta
! cat practice_fastas/NORW-22D87C_S81.primertrimmed.consensus.fa | sed '/^\s*$/d' | wc -l

#code snippet credit: https://stackoverflow.com/questions/114814/count-non-blank-lines-of-code-in-bash

2
1


# Merging fastas

MAFFT takes as input a single fasta file with as many sequences as need to be aligned.  As each of my files contains at most 1 sequence, I'll have to append them all together.

In [111]:
!  cat practice_fastas/*.fa > practice_fastas/merged-fasta.fa

Check that all 27 fastas made it in:

In [112]:
! grep -c ">" practice_fastas/merged-fasta.fa

27


Remove the empty records and check how many sequences there are now:

In [113]:
! awk 'BEGIN {RS = ">" ; FS = "\n" ; ORS = ""} $2 {print ">"$0}' practice_fastas/merged-fasta.fa > practice_fastas/merged-fasta-nonempty.fa
#code snippet credit: https://www.biostars.org/p/78786/

! grep -c ">" practice_fastas/merged-fasta-nonempty.fa
! head -1 practice_fastas/merged-fasta-nonempty.fa

22
>Consensus_COG111-PC_S86.primertrimmed.consensus_threshold_0.75_quality_20


Trees and things will be easier to read if the record names are shortened to just "COG111-PC_S86":

In [114]:
! sed -i -e 's/Consensus_//g' practice_fastas/merged-fasta-nonempty.fa
! sed -i -e 's/.primertrimmed.consensus_threshold_0.75_quality_20//g' practice_fastas/merged-fasta-nonempty.fa
! grep '>' practice_fastas/merged-fasta-nonempty.fa

>COG111-PC_S86
>COG112-PC_S114
>NORW-220299_S80
>NORW-22A50F_S96
>NORW-22AE40_S97
>NORW-22DDC8_S95
>NORW-22DFFF_S94
>NORW-23264D_S112
>NORW-23267A_S106
>NORW-2326A7_S131
>NORW-2326C5_S143
>NORW-2326F2_S128
>NORW-232829_S109
>NORW-232847_S115
>NORW-232883_S142
>NORW-232892_S118
>NORW-2328DE_S133
>NORW-232935_S135
>NORW-232971_S125
>NORW-232980_S124
>NORW-23299F_S140
>NORW-232BD5_S138


# Playing with MAFFT

Now try running MAFFT on autopilot, preserving the input order of sequences:

In [115]:
! mafft --inputorder practice_fastas/merged-fasta-nonempty.fa > practice_fastas/auto.fa

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 10552 ambiguous characters.
    1 / 22
done.

Constructing a UPGMA tree (efffree=0) ... 
   20 / 22
done.

Progressive alignment 1/2... 
STEP    21 / 21 
done.

Making a distance matrix from msa.. 
    0 / 22
done.

Constructing a UPGMA tree (efffree=1) ... 
   20 / 22
done.

Progressive alignment 2/2... 
STEP    21 / 21 
done.

disttbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more g

The output of MAFFT is a fasta file with the sequences aligned to each other.

In [116]:
! head practice_fastas/auto.fa

>COG111-PC_S86
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnctttcgatctcttgtagatct
gttctctaaacgaactnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnngnnnnnn
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnngctcgtacgtggctttgg
agactccgtggaggaggtcttatcagaggcacgtcaacatcttaaagatggcacttgtgg
cttagtagaagttgaaaaaggcgttttgcctcaacttgaacagccctatgtgttcatcaa
acgttcggatgctcgaactgcacctcatggtcatgttatggttgagctggtagcagaact


# Multiple alignment to a reference sequence

To align the sequences to the reference Wuhan-Wu-1, follow the syntax below (link: https://mafft.cbrc.jp/alignment/software/closelyrelatedviralgenomes.html).  The reference sequence for Wuhan-Wu-1 is available at: https://www.ncbi.nlm.nih.gov/nuccore/1798174254

In [117]:
# mafft --6merpair --addfragments othersequences referencesequence > output

In [118]:
! mafft --6merpair --addfragments practice_fastas/merged-fasta-nonempty.fa Wuhan-Wu-1.fasta > output.fa

nadd = 22
ppenalty_ex = -10
nthread = 0
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 1
njobc = 2
generating a scoring matrix for nucleotide (dist=200) ... done


Making a distance matrix ..

There are 10552 ambiguous characters
    1 / 1
done.

nambiguous = 0 / 29903, ratio=0.000000
fTEP 21 / 22                    

Combining ..
   done.                      

   done.                      

addsingle (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-fragment (Not tested.)
 ?

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.



In [119]:
! head output.fa

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
attaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct
gttctctaaacgaactttaaaatctgtgtggctgtcactcggctgcatgcttagtgcact
cacgcagtataattaataactaattactgtcgttgacaggacacgagtaactcgtctatc
ttctgcaggctgcttacggtttcgtccgtgttgcagccgatcatcagcacatctaggttt
cgtccgggtgtgaccgaaaggtaagatggagagccttgtccctggtttcaacgagaaaac
acacgtccaactcagtttgcctgttttacaggttcgcgacgtgctcgtacgtggctttgg
agactccgtggaggaggtcttatcagaggcacgtcaacatcttaaagatggcacttgtgg
cttagtagaagttgaaaaaggcgttttgcctcaacttgaacagccctatgtgttcatcaa
acgttcggatgctcgaactgcacctcatggtcatgttatggttgagctggtagcagaact


# Maximum likelihood tree

Now that you have the MSA, you can create a tree using IQTREE.  IQTREE exists as a [web server ](http://iqtree.cibiv.univie.ac.at/), and there is also a command line option.  I'm starting with the [Beginner's Tutorial](http://www.iqtree.org/doc/Tutorial).  Using the default parameters:

In [120]:
! iqtree -s output.fa -redo

IQ-TREE multicore version 1.6.12 for Linux 64-bit built Mar 23 2020
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams.

Host:    tamarisk (AVX2, FMA3, 9 GB RAM)
Command: iqtree -s output.fa -redo
Seed:    169559 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Fri Feb  4 12:11:10 2022
Kernel:  AVX+FMA - 1 threads (8 CPU cores detected)

HINT: Use -nt option to specify number of threads because your CPU has 8 cores!
HINT: -nt AUTO will automatically determine the best number of threads to use.

Reading alignment file output.fa ... Fasta format detected
Alignment most likely contains DNA/RNA sequences
Alignment has 23 sequences with 29905 columns, 244 distinct patterns
50 parsimony-informative, 57 singleton sites, 29798 constant sites
                  Gap/Ambiguity  Composition  p-value
   1  NC_045512.2         0.01%    passed     99.37%
   2  COG111-PC_S86       2.38%    passed    100.00%
   3  COG112-PC_S

Generating 98 parsimony trees... 0.062 second
Computing log-likelihood of 98 initial trees ... 0.111 seconds
Current best score: -41605.981

Do NNI search on 20 best initial trees
Estimate model parameters (epsilon = 0.100)
BETTER TREE FOUND at iteration 1: -41605.922
BETTER TREE FOUND at iteration 2: -41605.913
Iteration 10 / LogL: -41606.024 / Time: 0h:0m:0s
Iteration 20 / LogL: -41606.049 / Time: 0h:0m:0s
Finish initializing candidate tree set (20)
Current best tree score: -41605.913 / CPU time: 0.267
Number of iterations: 20
--------------------------------------------------------------------
|               OPTIMIZING CANDIDATE TREE SET                      |
--------------------------------------------------------------------
Estimate model parameters (epsilon = 0.100)
UPDATE BEST LOG-LIKELIHOOD: -41605.876
Iteration 30 / LogL: -41605.987 / Time: 0h:0m:0s (0h:0m:0s left)
Iteration 40 / LogL: -41605.960 / Time: 0h:0m:0s (0h:0m:0s left)
Estimate model parameters (epsilon = 0.100)
B

The maximum likelihood tree can be found as a drawing and in Newick format at the end of the .iqtree output file:

In [121]:
! cat output.fa.iqtree

IQ-TREE 1.6.12 built Mar 23 2020

Input file name: output.fa
Type of analysis: ModelFinder + tree reconstruction
Random seed number: 169559

REFERENCES
----------

To cite ModelFinder please use: 

Subha Kalyaanamoorthy, Bui Quang Minh, Thomas KF Wong, Arndt von Haeseler,
and Lars S Jermiin (2017) ModelFinder: Fast model selection for
accurate phylogenetic estimates. Nature Methods, 14:587–589.
https://doi.org/10.1038/nmeth.4285

To cite IQ-TREE please use:

Lam-Tung Nguyen, Heiko A. Schmidt, Arndt von Haeseler, and Bui Quang Minh
(2015) IQ-TREE: A fast and effective stochastic algorithm for estimating
maximum likelihood phylogenies. Mol Biol Evol, 32:268-274.
https://doi.org/10.1093/molbev/msu300

SEQUENCE ALIGNMENT
------------------

Input data: 23 sequences with 29905 nucleotide sites
Number of constant sites: 29798 (= 99.6422% of all sites)
Number of invariant (constant or ambiguous constant) sites: 29798 (= 99.6422% of all sites)
Number of parsimony informative sites: 50
Number o

In the output file for IQTREE, there is this note:
    
    WARNING: 12 near-zero internal branches (<0.0000) should be treated with caution
         Such branches are denoted by '**' in the figure below
         
This means that there are [polytomies](https://en.wikipedia.org/wiki/Polytomy) present in the data.  To (hopefully) resolve these, use TreeTime!

# TreeTime

TreeTime optimizes branch lengths and resolves polytomies by incorporating sampling date information into the tree building process.  The sample dates can be found in ________________.  The treetime source is [here](https://github.com/neherlab/treetime) and here's the [documentation](https://treetime.readthedocs.io/en/latest/index.html).

In [2]:
! treetime -h

usage: TreeTime: Maximum Likelihood Phylodynamics

positional arguments:
  {homoplasy,ancestral,mugration,clock,version}

optional arguments:
  -h, --help            show this help message and exit
  --tree TREE           Name of file containing the tree in newick, nexus, or
                        phylip format. If none is provided, treetime will
                        attempt to build a tree from the alignment using
                        fasttree, iqtree, or raxml (assuming they are
                        installed)
  --sequence-length SEQUENCE_LENGTH
                        length of the sequence, used to calculate expected
                        variation in branch length. Not required if alignment
                        is provided.
  --aln ALN             alignment file (fasta)
  --vcf-reference VCF_REFERENCE
                        only for vcf input: fasta file of the sequence the VCF
                        was mapped to.
  --dates DATES         csv file with dates for n

The inputs `treetime` requires are: a .csv of the sampling dates, the Newick tree from IQTREE, an outdir, and optionally the alignment from MAFFT.  Various formats of inputs can be used (see documentation).  The simplest implementation of treetime is:

In [ ]:
#! treetime --tree output.fa.treefile --dates data/h3n2_na/h3n2_na_20.metadata.csv --aln output.fa --outdir 20210513_timetree

The output is a "GTR model, a molecular clock model, and a time-stamped phylogeny".

If the evolutionary rate is hard to estimate from the data, you can specify the evolutionary rate using `--clock-rate <rate>`.  In the paper _________ the clock rate was _____.

**Coalescent priors in TreeTime**

A coalescent prior is optional, and can be added using `--coalescent <arg>`.  A coalescent prior is a time scale in units of time to divergence (time to coalescence, when viewed in reverse), given as a floating point number.  

According to [Wikipedia](https://en.wikipedia.org/wiki/Coalescent_theory), coalescent theory is about using models to estimate the time to coalescence (time backwards from two lineages to their most recent common ancestor) in evolutionary trees, including aspects like recombination, natural selection, gene flow, and other factors.  The time to coalescence is the number of preceding generations to coalescence (how many generations beforehand the most recent common ancestor was found).  In general, the expected time to coalescence, in units of generations, is equal to the population size (# copies of each locus).  A "coalescent event" is the place where two branches meet.

In [1]:
# ! treetime --tree data/ebola/ebola.nwk --dates data/ebola/ebola.metadata.csv --aln data/ebola/ebola.fasta --outdir ebola  --coalescent skyline